In [1]:
from __future__ import print_function, division
%pylab notebook
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from bokeh.plotting import *
output_notebook()

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

In [2]:
df = pd.read_csv('data/learning_resource_table_2016_07_05.csv')
df.head(2)

,id,title,subtitle,description,short_description,syllabus,url,slug,difficulty,created,...,available,end_date,typical_learning_time,rating,price,new,language,format,license,venue
0,cs101,Intro to Computer Science,Build a Search Engine & a Social Network,In this introduction to computer programming c...,Learn key computer science concepts in this in...,###Lesson 1: How to Get Started\n\n- Interview...,https://www.udacity.com/course/intro-to-comput...,intro-to-computer-science--cs101,beginner,NaN,...,NaN,NaN,3months,NaN,Free,NaN,en,MOOC,Creative Commons Attribution-NonCommercial- No...,Udacity
1,ud257,A/B Testing,Online Experiment Design and Analysis,This course will cover the design and analysis...,This course will cover the design and analysis...,### Lesson 1: Overview of A/B Testing\nThis le...,https://www.udacity.com/course/ab-testing--ud2...,ab-testing--ud257,intermediate,NaN,...,NaN,NaN,1months,NaN,Free,NaN,en,MOOC,Creative Commons Attribution-NonCommercial- No...,Udacity


In [3]:
print(df.columns)
print(len(df))

Index([u'id', u'title', u'subtitle', u'description', u'short_description',
       u'syllabus', u'url', u'slug', u'difficulty', u'created',
       u'date_modified', u'date_published', u'available', u'end_date',
       u'typical_learning_time', u'rating', u'price', u'new', u'language',
       u'format', u'license', u'venue'],
      dtype='object')
357


In [15]:
#df['all-text'] = df['title'] + ' ' + df['subtitle'] + ' ' + df['description'] + ' ' + df['short_description'] + ' ' + df['syllabus']
cols = ['title', 'description', 'syllabus']
df['all-text'] = ''
for i in cols:
    df['all-text'] += df[i].astype('str')
    df['all-text'] += ' '
df['all-text'].tail(10)

347    Data analytics involving text - VideoLectures....
348    Model-based Bayesian RL - VideoLectures.NET Al...
349    Optimization Algorithms in Machine Learning - ...
350    Training Deep Neural Networks - VideoLectures....
351    Tutorial on Learning Deep Architectures - Vide...
352    Adversarial Examples - VideoLectures.NET nan nan 
353    Mining Complex Entities from Heterogeneous Inf...
354    NLP and Deep Learning 2: Compositional Deep Le...
355    Scaling Up Deep Learning - VideoLectures.NET D...
356    Introduction to Graphical Models - VideoLectur...
Name: all-text, dtype: object

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm, tqdm_pandas
import string
allowed_chars = set(string.ascii_lowercase) | {' '}

stemmer = PorterStemmer()
def text_cleanup(text):
    #text = str(text)
    
    #text = text.replace('\n', ' ')
    text = text.lower()
    text = text.replace('videolectures.net', '')
    text = ' '.join((stemmer.stem(i.decode('utf-8')) for i in text.split()))
    text = text.replace('nan', '')
    text = filter(lambda x: x in allowed_chars, text.lower())
    return text
    
df['all-text'] = df['all-text'].apply(text_cleanup)

In [17]:
df['all-text'].tail(10)

347    data analyt involv text  data avail in electro...
348    modelbas bayesian rl  although bayesian method...
349    optim algorithm in machin learn  optim provid ...
350                         train deep neural network   
351    tutori on learn deep architectur  thi short tu...
352                                  adversari exampl   
353    mine complex entiti from heterogen inform netw...
354           nlp and deep learn  composit deep learn   
355    scale up deep learn  deep learn ha rapidli mov...
356                        introduct to graphic model   
Name: all-text, dtype: object

In [29]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.3, min_df=2)
vectorized_text = vectorizer.fit_transform(df['all-text'].values)
from sklearn.decomposition import TruncatedSVD
if False:
    svd = TruncatedSVD(n_components=500, n_iter=100)
    vectorized_text = svd.fit_transform(vectorized_text)
else:
    vectorized_text = np.array(vectorized_text.todense())
print(type(vectorized_text))
print(vectorized_text.shape)
print(len(vectorized_text.shape))

<type 'numpy.ndarray'>
(357, 1903)
2


In [30]:
from mvpa2.suite import *

In [31]:
#dim_range = 25
som = SimpleSOMMapper((20, 5), 400, learning_rate=0.5)
# n_rows, n_columns = dim_range, dim_range
# som = somoclu.Somoclu(n_columns, n_rows, data=vectorized_text.todense())
# som.train()

In [32]:
#som.view_umatrix(bestmatches=True, labels=df['title'])
som.train(vectorized_text)

In [33]:
binned_data = som(vectorized_text)

In [34]:
table = pd.DataFrame(columns=['x', 'y'], data=binned_data)
table['dummy'] = 1
table['title'] = df['title']
grp_table = table.groupby(by=('x', 'y'))
plot_table = pd.DataFrame(columns=['count'], data=grp_table.apply(lambda x: x['dummy'].sum()))
plot_table['titles'] = grp_table.apply(lambda x: ' || '.join([i[:60] for i in x['title'].values]))
plot_table = plot_table.reset_index()
plot_table.head(2)

,x,y,count,titles
0,0,1,6,Bioconductor for Genomic Data Science || Comma...
1,0,2,3,Introduction to Data Science || A Crash Course...


In [35]:
from bokeh.models import *
from bokeh.charts import HeatMap
from math import floor 

from bokeh.plotting import figure, output_file, show 
from bokeh.models import HoverTool, ColumnDataSource 
from bokeh.palettes import RdYlBu11 as palette 

x = plot_table['x']
y = plot_table['y']
c = plot_table['count']
t = plot_table['titles']
data = {'x':x, 'y':y, 'count':c, 'titles':t} 

# colormap by hand for now, a one-liner some day 
N, min, max = len(palette), c.min(), c.max() 
data['color'] = [] 
for x in data['count']: 
    ind = int(floor((x-min)/(max-min)*(N-1)))
    data['color'].append(palette[ind]) 

source = ColumnDataSource(data) 

hover = HoverTool( 
    #tooltips=[("description", "@titles"), ('count', '@count')] 
    tooltips = """
            <div>
                <span style="font-size: 10px; width: 200px; display: block;">@titles</span>
            </div>
            """
) 

hm = figure(title='test data', tools=[hover,"pan","box_zoom","wheel_zoom","reset","resize","save"]) 
hm.rect(x='x', y='y', width=1, height=1, fill_color='color', line_color="white", source=source) 
show(hm)

/opt/anaconda2/lib/python2.7/site-packages/ipykernel/comm/comm.py:52: DeprecationWarning: Comm._comm_id_default is deprecated: use @default decorator instead.
  def _comm_id_default(self):
/opt/anaconda2/lib/python2.7/site-packages/ipykernel/comm/comm.py:29: DeprecationWarning: Comm._iopub_socket_default is deprecated: use @default decorator instead.
  def _iopub_socket_default(self):
/opt/anaconda2/lib/python2.7/site-packages/ipykernel/comm/comm.py:24: DeprecationWarning: Comm._kernel_default is deprecated: use @default decorator instead.
  def _kernel_default(self):
/opt/anaconda2/lib/python2.7/site-packages/ipykernel/comm/comm.py:32: DeprecationWarning: Comm._session_default is deprecated: use @default decorator instead.
  def _session_default(self):
/opt/anaconda2/lib/python2.7/site-packages/ipykernel/comm/comm.py:41: DeprecationWarning: Comm._topic_default is deprecated: use @default decorator instead.
  def _topic_default(self):
/opt/anaconda2/lib/python2.7/site-packages/ipykerne

In [36]:
f_names = np.array(vectorizer.get_feature_names())
tags = f_names[vectorized_text.argmax(axis=1)]
print('tags:', tags)
print(len(tags))

tags: [u'comput' u'ab' u'visual' u'lesson' u'hadoop' u'realtim' u'panda'
 u'mapreduc' u'set' u'lesson' u'lesson' u'reinforc' u'pars' u'build'
 u'deep' u'lesson' u'lesson' u'click' u'product' u'account' u'discret'
 u'statist' u'machin' u'python' u'summer' u'genom' u'massiv' u'qualit'
 u'regress' u'busi' u'collection' u'answer' u'institut' u'econom'
 u'databas' u'biostatist' u'social' u'busi' u'capston' u'statist'
 u'exploratori' u'employe' u'product' u'mooc' u'recommend' u'tool'
 u'describ' u'sentiment' u'genom' u'market' u'format' u'variable' u'text'
 u'warehous' u'decisionmak' u'chines' u'artifici' u'abl' u'analyt'
 u'project' u'spreadsheet' u'dna' u'natur' u'assign' u'project' u'graph'
 u'infer' u'scienc' u'beauti' u'manag' u'languag' u'regress' u'regress'
 u'databases' u'scienc' u'peopl' u'ones' u'hadoop' u'discoveri' u'chapter'
 u'social' u'mani' u'genom' u'particip' u'represent' u'busi' u'statist'
 u'regress' u'statist' u'youll' u'life' u'warehous' u'predict' u'infer'
 u'project' 

In [37]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, learning_rate=100, n_iter=5000)

In [38]:
tsne_text = tsne.fit_transform(vectorized_text)

In [39]:
from bokeh.charts import Scatter

In [40]:
plot_df = pd.DataFrame(columns=['x', 'y'], data=tsne_text)
plot_df['title'] = df['title']
plot_df.head(2)

,x,y,title
0,3.550282,-1.922561,Intro to Computer Science
1,2.919194,-2.620359,A/B Testing


In [41]:
hover = HoverTool(
    tooltips=[("x", "@x"), ("x", "@y"), ("Titles", "@title")]
)

source = ColumnDataSource(plot_df)
p = figure(background_fill='#DFDFE5', plot_width=650, 
                          plot_height=450)
# Add the hover tool
p.add_tools(hover)

# Populate glyphs
p.circle(x='x', y='y', size=7, alpha=0.5, source=source)
#p = Scatter(plot_df, title="scat", width=800, tools=[hover, "pan","box_zoom","wheel_zoom","reset","resize","save"])
show(p)

/opt/anaconda2/lib/python2.7/site-packages/ipykernel/comm/comm.py:52: DeprecationWarning: Comm._comm_id_default is deprecated: use @default decorator instead.
  def _comm_id_default(self):
/opt/anaconda2/lib/python2.7/site-packages/ipykernel/comm/comm.py:29: DeprecationWarning: Comm._iopub_socket_default is deprecated: use @default decorator instead.
  def _iopub_socket_default(self):
/opt/anaconda2/lib/python2.7/site-packages/ipykernel/comm/comm.py:24: DeprecationWarning: Comm._kernel_default is deprecated: use @default decorator instead.
  def _kernel_default(self):
/opt/anaconda2/lib/python2.7/site-packages/ipykernel/comm/comm.py:32: DeprecationWarning: Comm._session_default is deprecated: use @default decorator instead.
  def _session_default(self):
/opt/anaconda2/lib/python2.7/site-packages/ipykernel/comm/comm.py:41: DeprecationWarning: Comm._topic_default is deprecated: use @default decorator instead.
  def _topic_default(self):
/opt/anaconda2/lib/python2.7/site-packages/ipykerne